In [1]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\D.Ivanovas\.cache\kagglehub\datasets\clmentbisaillon\fake-and-real-news-dataset\versions\1


In [2]:
import pandas as pd
import os 
fake_df = pd.read_csv(os.path.join(path, 'fake.csv'))
true_df = pd.read_csv(os.path.join(path, 'true.csv'))

fake_df["label"]=0
true_df["label"]=1

joined_df = pd.concat([fake_df, true_df], axis=0, ignore_index=True) # sujungiam sulablintus datasetus
joined_df = joined_df.sample(frac=1).reset_index(drop=True) # sushufflinam dataseta


# Text processing

In [ ]:
import spacy
import swifter
import re
nlp = spacy.load('en_core_web_sm')

# '\xad' soft hyphen | \u200e  left to right symbol | \u200a Unicode Character “ ” | \u200b Zero-width space
# \u200f Right-to-Left Mark
unwanted_symbols = ['–', '—', '‘', '’', '“', '”', '•', '…', '☑', '➡', 'ツ',  '¯','°', '´', '¿', '\xad', '\u200e', '\u200a', '\u200b', '\u200f']

def process(text):
    # replace all whitespaces, including unicode spaces and tabs with regular space
    text = re.sub(r'\s', ' ', text)
    doc = nlp(text)
    sent = []
    # check for unwanted symbols, stop words and punctuation
    for token in doc:
        if token.text not in unwanted_symbols and not token.is_punct and not token.is_stop:
            sent.append(token.text)
    
    return ' '.join(sent)

# use swifter for faster .apply()
joined_df['text'].swifter.apply(process)

IndentationError: expected an indented block after 'for' statement on line 13 (2873486216.py, line 14)

In [7]:
from spellchecker import SpellChecker
spell = SpellChecker()

misspelled_words = []

def check_misspell(text):
    misspelled = spell.unknown(text)
    misspelled_words.append(misspelled)
    
joined_df['text'].swifter.apply(check_misspell)

Pandas Apply:   0%|          | 0/44898 [00:00<?, ?it/s]

0        None
1        None
2        None
3        None
4        None
         ... 
44893    None
44894    None
44895    None
44896    None
44897    None
Name: text, Length: 44898, dtype: object

In [10]:
misspelled_words
a = [i for row in misspelled_words for i in row]

set(a)


{'\t',
 ' ',
 '  https://www.youtube.com/watch?v=yrxmfmgoptk',
 'https://www.youtube.com/watch?v=vkrctn0nevu',
 'read more: daily mail',
 '\xa0',
 '\xad',
 '¯',
 '°',
 '´',
 '¿',
 'à',
 'á',
 'â',
 'ç',
 'é',
 'ê',
 'í',
 'ñ',
 'ó',
 'ô',
 'ö',
 'ú',
 'ü',
 'ā',
 'ō',
 '\u200a',
 '\u200b',
 '\u200e',
 '\u200f',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '•',
 '…',
 '☑',
 '➡',
 'ツ',
 '️'}

In [14]:
import sys
sys.path.append(r'C:\Users\D.Ivanovas\Desktop\uni\Fake-News')

In [15]:
from paths import DATA_DIR
import shutil

shutil.move(str(path), str(DATA_DIR))

'C:\\Users\\D.Ivanovas\\Desktop\\uni\\Fake-News\\data'